참고

https://medium.com/analytics-vidhya/how-to-convert-tensorflow-object-detection-csv-data-to-coco-json-format-d0693d5b2f75

https://stackoverflow.com/questions/50916422/python-typeerror-object-of-type-int64-is-not-json-serializable

In [1]:
import pandas as pd
import os
import glob
import json
import cv2
import numpy as np

In [2]:
data = pd.read_csv("horse_data.csv")
path = "labeled-data/Sample20"
data_list = glob.glob(os.path.join(path,"*.png"))
data = data[data["image"].isin(data_list)].dropna().reset_index(drop=True)

In [3]:
df = pd.DataFrame()

df["x"] = 0
df["y"] = 0
df["w"] = 0
df["h"] = 0
df["width"] = 0
df["height"] = 0

In [4]:
x_cols = ['Nose_x', 'Eye_x', 'Nearknee_x', 'Nearfrontfetlock_x','Nearfrontfoot_x', 'Offknee_x',
          'Offfrontfetlock_x','Offfrontfoot_x','Shoulder_x','Midshoulder_x',
          'Elbow_x', 'Girth_x', 'Wither_x','Nearhindhock_x', 'Nearhindfetlock_x',
          'Nearhindfoot_x','Hip_x','Stifle_x', 'Offhindhock_x','Offhindfetlock_x','Offhindfoot_x',
          'Ischium_x']

y_cols = ['Nose_y', 'Eye_y', 'Nearknee_y', 'Nearfrontfetlock_y','Nearfrontfoot_y', 'Offknee_y',
          'Offfrontfetlock_y','Offfrontfoot_y','Shoulder_y','Midshoulder_y',
          'Elbow_y', 'Girth_y', 'Wither_y','Nearhindhock_y', 'Nearhindfetlock_y',
          'Nearhindfoot_y','Hip_y','Stifle_y', 'Offhindhock_y','Offhindfetlock_y','Offhindfoot_y',
          'Ischium_y']

In [5]:
for i in range(len(data)):
    xmax = data.iloc[i][x_cols].max()
    xmin = data.iloc[i][x_cols].min()
    
    ymax = data.iloc[i][y_cols].max()
    ymin = data.iloc[i][y_cols].min()
    
    img = cv2.imread(data.iloc[i]["image"])
    
    w = img.shape[0]
    h = img.shape[1]
    
    df.loc[i] = [xmin, ymax, xmax-xmin, ymax-ymin, w, h]

In [6]:
data = pd.concat([data, df], axis=1)

In [ ]:
data.to_csv("horse.csv",index=False)

In [7]:
images = []
categories = []
annotations = []

category = {}
category["supercategory"] = 'animal'
category["id"] = 1
category["name"] = 'horse'
categories.append(category)

data['fileid'] = data['image'].astype('category').cat.codes
data['categoryid']= 1
data['annid'] = data.index

In [8]:
def image(row):
    image = {}
    image["height"] = row.height
    image["width"] = row.width
    image["id"] = row.fileid
    image["file_name"] = row.image
    return image

def category(row):
    category = {}
    category["supercategory"] = "animal"
    category["id"] = row.categoryid
    category["name"] = "horse"
    return category

def make_keypoints(row):
    row = row[1:]
    key = []
    for i in range(44):
        if (i!=0 and i%2==0):
            key.append(2)
        key.append(row[i])
    key.append(2)
        
    return key

def annotation(row):
    annotation = {}
    area = row.w * row.h
    annotation["segmentation"] = []
    annotation["iscrowd"] = 0
    annotation["area"] = area
    annotation["image_id"] = row.fileid

    annotation["bbox"] = [row.x, row.y, row.w, row.h]

    annotation["category_id"] = row.categoryid
    annotation["id"] = row.annid
    
    annotation["keypoints"] = make_keypoints(row)
    annotation["num_keypoints"] = 22
    return annotation

In [9]:
for i in range(len(data)):
    annotations.append(annotation(data.loc[i]))
    images.append(image(data.loc[i]))
    categories.append(category(data.loc[i]))

In [10]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [11]:
data_coco = {}
data_coco["images"] = images
data_coco["categories"] = categories
data_coco["annotations"] = annotations

json.dump(data_coco, open("horse.json", "w"),cls=NpEncoder)

In [12]:
with open("horse.json", "r") as json_file:
    json_load = json.load(json_file)